# Knowledge Distillation

Knowledge distillation is a process in which a large, complex model is used to teach a smaller, simpler model. The idea is to transfer the knowledge learned by the large model to the smaller model, so that the smaller model can perform well on a given task while using fewer resources. This can be especially useful for tasks that require real-time processing or that need to be performed on devices with limited computational power. The process of knowledge distillation involves training the smaller model to mimic the behavior of the larger model, often by using the output of the larger model as a "teacher" to guide the training of the smaller model.

Here are some general steps to create a knowledge distillation model:

1. Train a large, complex model on the task of interest. This model will serve as the "teacher" for the knowledge distillation process.

2. Generate a set of training data for the knowledge distillation model. This data should be representative of the task that the model will be used for.

3. Train a smaller, simpler model on the same task using the training data generated in step 2.

4. During training, use the output of the large model as a "teacher" to guide the training of the smaller model. This is done by minimizing the difference between the output of the large model and the output of the small model.

5. Fine-tune the smaller model on the task of interest, using the same training data as in step 2.

6. Evaluate the performance of the knowledge distillation model on a set of test data to determine how well it performs on the task of interest.

It's worth noting that the exact steps for creating a knowledge distillation model can vary depending on the specific task and models being used. There are many resources available online that provide more detailed guidance on this topic.

Now I will use MNIST dataset as sample to create a knowledge distillation model.

## Import Packages

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

## Create Distiller Class

In [2]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            # Compute scaled distillation loss from https://arxiv.org/abs/1503.02531
            # The magnitudes of the gradients produced by the soft targets scale
            # as 1/T^2, multiply them by T^2 when using both hard and soft targets.
            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

## Create student and teacher models

In [3]:
# Create the teacher
teacher = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(512, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="teacher",
)

# Create the student
student = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(16, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="student",
)

# Clone student for later comparison
student_scratch = keras.models.clone_model(student)

## Prepare the dataset

In [4]:
# Prepare the train and test dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize data
x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 28, 28, 1))

11490434/11490434 [==============================] - 1s 0us/step


## Train the teacher

In [5]:
# Train teacher as usual
teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate teacher on data.
teacher.fit(x_train, y_train, epochs=10)
teacher.evaluate(x_test, y_test)

Epoch 1/10
1875/1875 [==============================] - 17s 5ms/step - loss: 0.1450 - sparse_categorical_accuracy: 0.9562
Epoch 2/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0934 - sparse_categorical_accuracy: 0.9716
Epoch 3/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0813 - sparse_categorical_accuracy: 0.9762
Epoch 4/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0740 - sparse_categorical_accuracy: 0.9786
Epoch 5/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0700 - sparse_categorical_accuracy: 0.9803
Epoch 6/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0652 - sparse_categorical_accuracy: 0.9823
Epoch 7/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0654 - sparse_categorical_accuracy: 0.9831
Epoch 8/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0628 - sparse_categorical_accuracy: 0.9838
Epoch 9/10
1875/1875 [==

[0.14949066936969757, 0.9753999710083008]

In [6]:
teacher.summary()

Model: "teacher"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 256)       2560      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 14, 14, 256)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 256)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 512)         1180160   
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 10)                250890    
                                                           

In [7]:
student.summary()

Model: "student"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 14, 14, 16)        160       
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 14, 14, 16)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 16)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 32)          4640      
                                                                 
 flatten_1 (Flatten)         (None, 1568)              0         
                                                                 
 dense_1 (Dense)             (None, 10)                15690     
                                                           

## Distill teacher to student

In [8]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 9s 4ms/step - sparse_categorical_accuracy: 0.8654 - student_loss: 1.1450 - distillation_loss: 24.9329
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - sparse_categorical_accuracy: 0.9546 - student_loss: 0.3343 - distillation_loss: 6.7705
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - sparse_categorical_accuracy: 0.9678 - student_loss: 0.2122 - distillation_loss: 3.9092
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - sparse_categorical_accuracy: 0.9717 - student_loss: 0.1809 - distillation_loss: 3.0789
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - sparse_categorical_accuracy: 0.9738 - student_loss: 0.1622 - distillation_loss: 2.6220
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - sparse_categorical_accuracy: 0.9753 - student_loss: 0.1506 - distillation_loss: 2.3036
Epoch 7/10
1875/1875 [==============================] - 7

In [9]:
student.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
student.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.1756 - sparse_categorical_accuracy: 0.9733


[0.17564243078231812, 0.9732999801635742]

## Train student from scratch for comparison

In [10]:
# Train student as doen usually
student_scratch.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch.fit(x_train, y_train, epochs=10)
student_scratch.evaluate(x_test, y_test)

Epoch 1/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2155 - sparse_categorical_accuracy: 0.9356
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0899 - sparse_categorical_accuracy: 0.9718
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0741 - sparse_categorical_accuracy: 0.9772
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0638 - sparse_categorical_accuracy: 0.9801
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0586 - sparse_categorical_accuracy: 0.9817
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0530 - sparse_categorical_accuracy: 0.9833
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0496 - sparse_categorical_accuracy: 0.9840
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0450 - sparse_categorical_accuracy: 0.9863
Epoch 9/10
1875/1875 [==========

[0.05867832154035568, 0.9822999835014343]

## Conclusion
The test accuracy for teacher model, student model using Knowledege Distillation, student model trained from scratch are about 0.976, 0.978, 0.98. Teacher model have 1.47 million parameters. Student model has 20000 parameters. By training with Knowledeg Distillation technology, student model can get similar test accuracy compared to teacher model with 1.3% model size.


## References
* [Distilling the Knowledge in a Neural Network](https://arxiv.org/abs/1503.02531)
* [Keras Example: Knowledge Distillation](https://keras.io/examples/vision/knowledge_distillation/)